## Introduction
https://www.tutorialspoint.com/flask/flask_wtf.htm  

* One of the essential aspects of a web application is to present a user interface for the user. HTML provides a `<form>` tag, which is used to design an interface. A Form’s elements such as text input, radio, select etc. can be used appropriately.

* In the form of **Http request message**, data entered by a user is submitted to the server side script by either GET or POST method.

* The Server side script has to recreate the form elements from http request data. So in effect, form elements have to be defined twice – once in HTML and again in the server side script.

* Another disadvantage of using **HTML form** is that it is difficult (if not impossible) to render the form elements dynamically. HTML itself provides no way to validate a user’s input.

* This is where WTForms, a flexible form, rendering and validation library comes handy. Flask-WTF extension provides a simple interface with this WTForms library.* 

* Using Flask-WTF, we can define the form fields in our Python script and render them using an HTML template. It is also possible to apply validation to the WTF field.

## Section 9: Forms with Flask 

### 00-Basic-Flask-Form.py

In [ ]:
#00-Basis-Flask-Form.py
from flask import Flask, render_template
from flask_wtf import FlaskForm
from wtforms import StringField,SubmitField
#The above line import the fields we want to use in the Form. 

app = Flask(__name__)

# Configure a secret SECRET_KEY
# We will later learn much better ways to do this!!
app.config['SECRET_KEY'] = 'mysecretkey'

# Now create a WTForm Class
# Lots of fields available:
# http://wtforms.readthedocs.io/en/stable/fields.html
# The class definition below is usually put into a separate py file, particularly when the application becomes large.
class InfoForm(FlaskForm):
    '''
    This general class gets a lot of form about puppies.
    Mainly a way to go through many of the WTForms Fields.
    '''
    breed = StringField('What breed are you?')
    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST']) # The 'methods = ...' will pass the form into templates. 
def index():
    breed = False
    form = InfoForm()
    if form.validate_on_submit():
        # Grab the data from the breed on the form.
        breed = form.breed.data # Note the breed here is different from the breed in InfoForm defined earlier. 
        # Reset the form's breed data to be False (should be ''? Empty string is False?).
        # Empty strings are "falsy" which means they are considered false in a Boolean context. This is from Internet.
        form.breed.data = '' #We can also set it as False. If so, everytime, after submitting, it will show False in the form. 
    return render_template('00-home.html', form=form, breed=breed)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#00-home.html
<p>
{% if breed %} #See the py file earlier. The default value for breed is False and so we use else branch. However, if not...
  The breed you entered is {{breed}}.
  You can update it in the form below:
{% else %}
  Please enter your breed in the form below:
{% endif %}
</p>

#Without the following, no form will show up in the web page
<form method="POST">
    {# This hidden_tag is a CSRF security feature. http://flask-wtf.readthedocs.io/en/stable/csrf.html #}
  
    {{ form.hidden_tag() }} #Although we don't define .hidden_tag() in our form class earlier, we inherit it from FlaskForm
    {{ form.breed.label }} {{ form.breed(class='some-css-class') }}
    {{ form.submit() }}
</form>

### 01-Form-Fields.py

In [ ]:
from flask import Flask, render_template, session, redirect, url_for, session
#Note here we have the url_for in python script. We use url_for in the template side earlier.  
from flask_wtf import FlaskForm
from wtforms import (StringField, BooleanField, DateTimeField,
                     RadioField,SelectField,TextField,
                     TextAreaField,SubmitField)
#When you import multilines, using () to prevent error

from wtforms.validators import DataRequired


app = Flask(__name__)
# Configure a secret SECRET_KEY
# We will later learn much better ways to do this!!
app.config['SECRET_KEY'] = 'mysecretkey'

class InfoForm(FlaskForm):
  
    breed = StringField('What breed are you?',validators=[DataRequired()])
    neutered  = BooleanField("Have you been neutered?")
    mood = RadioField('Please choose your mood:', choices=[('mood_one','Happy'),('mood_two','Excited')])
    food_choice = SelectField(u'Pick Your Favorite Food:',
                          choices=[('chi', 'Chicken'), ('bf', 'Beef'),
                                   ('fish', 'Fish')])
    # u before u'Pick is for unicode. This is to prevent error in some systems.
    feedback = TextAreaField()
    submit = SubmitField('Submit')

@app.route('/', methods=['GET', 'POST'])
def index():

    form = InfoForm()
    if form.validate_on_submit():
        # Grab the data from the breed on the form.
        # session is for temporarily hold data for the user-login session. It is not permanent holding of data as database. 
        session['breed'] = form.breed.data 
        session['neutered'] = form.neutered.data
        session['mood'] = form.mood.data
        session['food'] = form.food_choice.data
        session['feedback'] = form.feedback.data

        return redirect(url_for("thankyou")) # In FlaskBasics and Template, we use url_for in the template. 
        #This return is for the if statement 'if form.validate_on_submit():
        #This means if user submit form, then we will not use the 'return render_template(....)?


    return render_template('01-home.html', form=form)


@app.route('/thankyou')
def thankyou():
    return render_template('01-thankyou.html')

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#01-home.html
<h1>Welcome to Puppy Surveys</h1>
<form  method="POST"> #Here we don't need <form class =...> because we use WTF form but not html Form. 
    {# This hidden_tag is a CSRF security feature. #}
    {{ form.hidden_tag() }}
    {{ form.breed.label }} {{form.breed}}
    <br> #break line. 
    {{ form.neutered.label}} {{form.neutered}}
    <br>
    {{form.food_choice.label}}{{form.food_choice}}
    <br>
    {{form.mood.label}}{{form.mood}}
    <br>
    Any other feedback?
    <br>
    {{form.feedback}}
   <br>
    {{ form.submit() }}
</form>

In [ ]:
# 01-thankyou.html
<h1>Thank you. Here is the info you gave:</h1>
<ul>
  <li>Breed: {{session['breed']}}</li>
  <li>Neutered: {{session['neutered']}}</li>
  {# Note, this saves the mood key, not the form value!!! #}
  <li>Mood: {{session['mood']}}</li>
  <li>Food: {{session['food']}}</li>
  <li>Feedback: {{session['feedback']}}</li>
</ul>

### 03-Flashing-Messages.py
* If not clear about the Bootstrap code in template file, watch the Video for how to create a button with 'dismiss' action.
* The above code actually include 02 and 03 part in the course. 03 include 02. 
* Better run the code before examining. 

In [ ]:
from flask import Flask, render_template, session, redirect, url_for, session, flash
from flask_wtf import FlaskForm
from wtforms import (StringField, BooleanField, DateTimeField,
                     RadioField,SelectField,TextField,
                     TextAreaField,SubmitField)
from wtforms.validators import DataRequired

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mysecretkey'

class InfoForm(FlaskForm):
    breed = StringField('What breed are you?')
    submit = SubmitField('Submit')


@app.route('/', methods=['GET', 'POST'])
def index():
    form = InfoForm()
    # If the form is valid on submission (we'll talk about validation next)
    if form.validate_on_submit():
        # Grab the data from the breed on the form.

        session['breed'] = form.breed.data
        flash(f"You just changed your breed to: {session['breed']}")
        return redirect(url_for("index"))
    return render_template('03-home.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#03-home.html
<head>
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>

</head>


{# Source for this bootstrapcode:
https://getbootstrap.com/docs/4.0/components/alerts/#dismissing #}
<div class="container">

  {# get_flashed_messages() is auto sent to the template with the flash() call in the .py file #}
      {% for mess in get_flashed_messages()  %} 
      #Here we use for loop because there might be several flash calls. However, even just one call, we can still use for loop.
      <div class="alert alert-warning alert-dismissible fade show" role="alert">
        <button type="button" class="close" data-dismiss="alert" aria-label="Close" class="fade close">
          <span aria-hidden="true">&times;</span>
        </button>
        {{mess}}
        </div>
      {% endfor %}


<form method="POST">
    {# This hidden_tag is a CSRF security feature. #}
    {{ form.hidden_tag() }}
    {{ form.breed.label }} {{ form.breed() }}
    {{ form.submit() }}
</form>
</div>

## Section 10: SQL databases with Flask
* Flask-SQLAlchemy and others have already been installed as it is specified in the requirements.txt 
* With some libraries like SQLAlchemy, we may just develop our Flask application without real SQL statements. 
* To connect SQL python, we need an ORM (object relational mapper) that converts python code to SQL statements. The most common ORM for Python is the aforementioned SQLAlchemy. 
* Flask-SQLAlchemy is an extension to SQLAlchemy that allows an easy connection between Flask and SQLAlchemy.  
* We can also write directly SQL statements as strings, and then use a way to connect to SQL. See other notes. 

### BasicModelApp.py

In [ ]:
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# This grabs the directory of current file
basedir = os.path.abspath(os.path.dirname(__file__))
# We will create database in this directory

app = Flask(__name__)
# SET UP OUR SQLite DATABASE 
# Connects our Flask App to our Database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
# With the help of ORM, we create our database with a few lines above. 


# Let's create our first model. Here model refers to a table. 
# We inherit from db.Model class
class Puppy(db.Model):

    # If you don't provide this, the default table name will be the class name
    # So the following actually manually override with a different table name. 
    __tablename__ = 'puppies'

    # create columns
    # Primary Key column, unique id for each puppy
    id = db.Column(db.Integer,primary_key=True)
    # Puppy name
    name = db.Column(db.Text)
    # Puppy age in years
    age = db.Column(db.Integer)

    # This sets what an instance in this table will have
    # Note the id will be auto-created for us later, so we don't add it here!
    def __init__(self,name,age):
        self.name = name
        self.age = age

    def __repr__(self):
        # This is the string representation of a puppy in the model
        return f"Puppy {self.name} is {self.age} years old."

### SetupDatabase.py
* If you run this script multiple times you will add multiple puppies to the database. That is okay, just the ids will be higher than 1, 2 on the subsequent runs 

In [ ]:
from BasicModelApp import db, Puppy

# Create the tables in the database 
db.create_all()

# Create new entries in the database
sam = Puppy('Sammy',3)
frank = Puppy('Frankie',4)

# Check ids (haven't added sam and frank to database table, so they should be None)
print(sam.id)
print(frank.id)

# Ids will get created automatically once we add these entries to the DB
db.session.add_all([sam,frank])

# Alternative for individual additions:
# db.session.add(sam)
# db.session.add(frank)

# Now save it to the database. Before this, entries might be in some temporary place. 
db.session.commit()

# Check the ids
print(sam.id)
print(frank.id)
# The ids will no longer be None, as database automatically create them when we commit two records. 

### basicCRUD.py
Now that the table has been created by running BasicModelApp and SetUpDatabase, we can play around with CRUD commands.
This is just an overview, usually we won't run a single script like this. Our goal here is to just familiarize ourselves with CRUD commands. 

In [ ]:
from BasicModelApp import db,Puppy

# We don't have to run db.create_all() as the database table has already been created. 

###### CREATE ############
my_puppy = Puppy('Rufus',5)
db.session.add(my_puppy)
db.session.commit()

###### READ ##############
# Note lots of ORM filter options here.
# filter(), filter_by(), limit(), order_by(), group_by()
# Also lots of executor options
# all(), first(), get(), count(), paginate()

all_puppies = Puppy.query.all() # list of all puppies objects in table
print(all_puppies)
print('\n')
# Grab by id
puppy_one = Puppy.query.get(1)
print(puppy_one)
print(puppy_one.age)
print('\n')
# Filters
puppy_sam = Puppy.query.filter_by(name='Sammy') # Returns list
print(puppy_sam)
print('\n')

###### UPDATE ############
# Grab your data, then modify it, then save the changes.
first_puppy = Puppy.query.get(1)
first_puppy.age = 10
db.session.add(first_puppy)
db.session.commit()

###### DELETE ############
second_pup = Puppy.query.get(2)
db.session.delete(second_pup)
db.session.commit()


# Check for changes:
all_puppies = Puppy.query.all() # list of all puppies in table
print(all_puppies)

### Flask Migrate
* Sometimes you may want add a column to your created table in python script. However, updating this in python does not automatically update the table in the database. These are called migrate changes to the database table. 
* We could use Flask-Migrate(already installed) to do this. There are four main commands used in the command line to perform the migration.
    * Set FLASK_APP environment variable by **'set FLASK_APP=BasicModelApp.py'**. Note this is just for Flask to know what is your application file. This has nothing to do with the file path. If we did not run the set up above, then Flask will automatically find an application called app.py.  
    *  We first set up our migrations directory by running: **'flask db init'**. This creates a migration repository. You only run this command once at the start of you using migrations to manage changes to your database. You should now see a migrations folder with a versions folder. Now we will be able to keep track of changes to the database in case we later decide to update our models. 
    * Now that the folder has been created and set up, we can create pending migrations based on our current model. Right now our .py file is called BasicModelApp.py, but for larger projects later on we will have this file simply be called models.py. It is time to now use the migrate command, this will create a unique id for the migration. Typically this unique id is just a number code like a437f282d3 , which isn't helpful for trying to remember past changes. So we can add a -m flag which allows us to add a message to our unique id. The next command then will look like this for us: **flask db migrate -m "created puppy table"**. 
    * Keep in mind this command is not actually running this migration or updating our database based on model changes. Its just created a new file that can later be used to update the database. To run the pending migration just created, we use the upgrade command. Which looks like this: **flask db upgrade** 
    
* The above process is very similar to the git commands system. The final one is just like commit command. 
    
* **Comments: In fact, if the created database is small and easy to create again, we can just delete it and then create it again. This way, we don't have to use the above commands.**

### BasicModelApp.py
* A few changes have been made to the formally created BasicModelApp.py. First add 'from flask_migrate import Migrate', and add 'Migrate(app,db)' to let the app have the migration capability. Second we add a new column name 'breed' to the model (table). We want to update this changes in .py file to the database (the .sqlite file created before) with the migration commands introduced before. 
* Run the four commands before, and then we will upgrade the changes. Note this file should put in the same folder as the database file. 

In [ ]:
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate  #This is newly added to have migration capability. 
######################################
#### SET UP OUR SQLite DATABASE #####
####################################

# This grabs our directory
basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
# Connects our Flask App to our Database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

Migrate(app,db) #This is newly added to have migration capability. 

#####################################
####################################
###################################


class Puppy(db.Model):

    __tablename__ = 'puppies'

    id = db.Column(db.Integer,primary_key=True)
    name = db.Column(db.Text)
    age = db.Column(db.Integer)
    breed = db.Column(db.Text) #This is newly added. Thus we want to update the database. 

    def __init__(self,name,age,breed):
        self.name = name
        self.age = age
        self.breed = breed

    def __repr__(self):

        return f"Puppy {self.name} is {self.age} years old."

### models. py
* This is a new project for studying the relationship of different database models/tables. It is not connected to previous cells. 
* Running steps: 
    * In the 02-Relationship folder, delete everything except two files: models.py and populate_database.py.
    * Run the following four commands for migration: (1) set FLASK_APP=models.py. Note I just need set environment variable for this file but not populate_datebase.py. (2) 'flask db init'. After this we should see the migration folder. This is like initiating a git repository. (3) 'flask db migrate -m "initial migration". This is like staging files in git? (4) 'flask db upgrade'. This is like 'commit' in git. 
    * Without running those commands above, we cannot run the script because in the code there are script for migration. 
    * Run: python populate_database.py. 


In [ ]:
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
######################################
#### SET UP OUR SQLite DATABASE #####
####################################

# This grabs our directory
basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
# Connects our Flask App to our Database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app,db)


class Puppy(db.Model):

    __tablename__ = 'puppies'

    id = db.Column(db.Integer,primary_key = True)
    name = db.Column(db.Text)
    # This is a one-to-many relationship
    # A puppy can have many toys
    toys = db.relationship('Toy',backref='puppy',lazy='dynamic')
    # This is a one-to-one relationship
    # A puppy only has one owner, thus uselist is False.
    # Strong assumption of 1 dog per 1 owner and vice versa.
    owner = db.relationship('Owner',backref='puppy',uselist=False)

    def __init__(self,name):
        # Note how a puppy only needs to be initalized with a name!
        self.name = name


    def __repr__(self):
        if self.owner:
            return f"Puppy name is {self.name} and owner is {self.owner.name}"
        else:
            return f"Puppy name is {self.name} and has no owner assigned yet."

    def report_toys(self):
        print("Here are my toys!")
        for toy in self.toys:
            print(toy.item_name)
        

class Toy(db.Model):

    __tablename__ = 'toys'

    id = db.Column(db.Integer,primary_key = True)
    item_name = db.Column(db.Text)
    # Connect the toy to the puppy that owns it.
    # We use puppies.id because __tablename__='puppies'
    puppy_id = db.Column(db.Integer,db.ForeignKey('puppies.id'))

    def __init__(self,item_name,puppy_id):
        self.item_name = item_name
        self.puppy_id = puppy_id


class Owner(db.Model):

    __tablename__ = 'owners'

    id = db.Column(db.Integer,primary_key= True)
    name = db.Column(db.Text)
    # We use puppies.id because __tablename__='puppies'
    puppy_id = db.Column(db.Integer,db.ForeignKey('puppies.id'))

    def __init__(self,name,puppy_id):
        self.name = name
        self.puppy_id = puppy_id

### populate_database.py
This script will call the models.py and create some puppies, owners, and toys!  
Note, if you run this more than once, you'll be creating dogs with the same name and duplicate owners. The script will still work, but you'll see some warnings. However, I can always start from very beginning by deleting migration folder and database file .sqlite. See running steps in the previous cells. 

In [ ]:
from models import db,Puppy,Owner,Toy

# Create 2 puppies
rufus = Puppy("Rufus")
fido = Puppy("Fido")

# Add puppies to database
db.session.add_all([rufus,fido])
db.session.commit()

# Check with a query, this prints out all the puppies!
print(Puppy.query.all())

# Grab Rufus from database
# Grab all puppies with the name "Rufus", returns a list, so index [0]
# Alternative is to use .first() instead of .all()[0]
rufus = Puppy.query.filter_by(name='Rufus').all()[0]

# Create an owner to Rufus
jose = Owner("Jose",rufus.id)

# Give some Toys to Rufus
toy1 = Toy('Chew Toy',rufus.id)
toy2 = Toy("Ball",rufus.id)

# Commit these changes to the database
db.session.add_all([jose,toy1,toy2])
db.session.commit()

# Let's now grab rufus again after these additions
rufus = Puppy.query.filter_by(name='Rufus').first()
print(rufus)

# Show toys
print(rufus.report_toys())

# You can also delete things from the database:
# find_pup = Puppy.query.get(1)
# db.session.delete(find_pup)
# db.session.commit()


**Not how the relation of tables are implemented. The relations are there but no joined displayed table is obtained as in SQL**

### adoption_site.py
This is a project from scratch for a puppy adoption site.  Here are structure and steps. 
* Due to the existing database file and migration folder from previous running, we need first delete all of them except the template folder and two py files: adoption_site.py and forms.py. Note to build a website from scratch, this is also the basic structures we usually need. 
* The subject of this project in udemy is called 'Database in views Part 1/2/3'. Note views is just the view function, usually corresponding to a html template.  
* The fundamental files we need for a webiste include those for display or list items in database, add to or delete from database. 
* Before running the script I need run the following four commands for migration: 
    * set FLASK_APP=adoption_site.py
    * flask db init
    * flask db migrate -m "message"
    * flask db upgrade

In [ ]:
import os
from forms import  AddForm , DelForm
from flask import Flask, render_template, url_for, redirect
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
app = Flask(__name__)
# Key for Forms
app.config['SECRET_KEY'] = 'mysecretkey'

############################################

        # SQL DATABASE AND MODELS. For big webiste, we usually put database stuff in a separate models.py. 

##########################################
basedir = os.path.abspath(os.path.dirname(__file__))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app,db)

class Puppy(db.Model):

    __tablename__ = 'puppies'
    id = db.Column(db.Integer,primary_key = True)
    name = db.Column(db.Text)

    def __init__(self,name):
        self.name = name

    def __repr__(self):
        return f"Puppy name: {self.name}"

############################################

        # VIEWS WITH FORMS

##########################################
@app.route('/')
def index():
    return render_template('home.html')

@app.route('/add', methods=['GET', 'POST'])
def add_pup():
    form = AddForm()

    if form.validate_on_submit():
        name = form.name.data

        # Add new Puppy to database
        new_pup = Puppy(name)
        db.session.add(new_pup)
        db.session.commit()

        return redirect(url_for('list_pup'))

    return render_template('add.html',form=form) #This is the default page when form.validate_on_submit() is false.

@app.route('/list')
def list_pup():
    # Grab a list of puppies from database.
    puppies = Puppy.query.all()
    return render_template('list.html', puppies=puppies)

@app.route('/delete', methods=['GET', 'POST'])
def del_pup():

    form = DelForm()

    if form.validate_on_submit():
        id = form.id.data
        pup = Puppy.query.get(id)
        db.session.delete(pup)
        db.session.commit()

        return redirect(url_for('list_pup'))
    return render_template('delete.html',form=form)


if __name__ == '__main__':
    app.run(debug=True)


### forms.py

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, IntegerField, SubmitField



class AddForm(FlaskForm):

    name = StringField('Name of Puppy:')
    submit = SubmitField('Add Puppy')

class DelForm(FlaskForm):

    id = IntegerField('Id Number of Puppy to Remove:')
    submit = SubmitField('Remove Puppy')

### base.html

In [ ]:
<!DOCTYPE html>
<html lang="en" dir="ltr">
  <head>
    <meta charset="utf-8">
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>

    <title>Adoption</title>
  </head>
  <body>

    <nav class="navbar navbar-expand-lg navbar-light bg-light">

    <div class="navbar-nav">
      <a class="nav-item nav-link" href="{{ url_for('index') }}">Home</a>
      <a class="nav-item nav-link" href="{{ url_for('add_pup') }}">Add Pup</a>
      <a class="nav-item nav-link" href="{{ url_for('list_pup') }}">List of Pups</a>
      <a class="nav-item nav-link" href="{{ url_for('del_pup') }}">Delete Pup</a>

    </div>

</nav>

    {% block content %}

    {% endblock %}
  </body>
</html>


### home.html

In [ ]:
{% extends "base.html" %}
{% block content %}
<div class="jumbotron">
  <h1>Welcome to our Adoption Page</h1>
  <p>Please select one of the links from the nav bar.</p>
</div>
{% endblock %}

### add.html

In [ ]:
{% extends "base.html" %}
{% block content %}
<div class="jumbotron">
  <h1>Have a puppy that needs to be adopted?</h1>
  <p>Add a new puppy with the form below:</p>
  <form method="POST">
      {# This hidden_tag is a CSRF security feature. #}
      {{ form.hidden_tag() }}
      {{ form.name.label }} {{ form.name() }}
      {{ form.submit() }}
  </form>
</div>
{% endblock %}

### delete.html

In [ ]:
{% extends "base.html" %}
{% block content %}
<div class="jumbotron">
  <h1>Did a puppy get adopted?</h1>
  <p>Fill out the form to remove the puppy from the list.</p>
  <form method="POST">
      {# This hidden_tag is a CSRF security feature. #}
      {{ form.hidden_tag() }}
      {{ form.id.label }} {{ form.id() }}
      {{ form.submit() }}
  </form>
</div>
{% endblock %}


### list.html

In [ ]:
{% extends "base.html" %}
{% block content %}
<div class="jumbotron">
  <h1>Here is a list of all available puppies.</h1>
  <ul>
    {% for pup in puppies  %}
    <li>{{pup}}</li>
    {% endfor %}
  </ul>

</div>
{% endblock %}

**The above project is in the 03-databases-in-views folder. There is another project in the 04-Project-solutions folder, which is a similar project but with a 'add owner' tab in the web.** However, the more formal one which include everything is in the following cell.

## Section 11: Large Flask Applications
### Running Steps

* Note always need activate flask virtual environment first. 
* Find file at 'courseNotes\web\Flask\06-Larger-Flask-Applications\00-structure-example.txt' and see the structure of a large Flask applications. 
* Check the project folder structure in C:\Users\ljyan\Desktop\courseNotes\web\Flask\06-Larger-Flask-Applications\01-Using-Blueprints, and then study the following steps if necessary. **Also be sure to understand the running flow chart of this application**. See the comments in those files about this. 
* The general structure is like this. 
    * A simple start up file called app.py.
    * Within the same folder where app.py located, we also include the system installation requirements. This is just for deployment. Note the migrations folder is created by system for database migration. 
    * The key structure is the myproject folder which is in the same level as app.py. 
        * Within the myproject folder, we have many models or tables for such as puppies and owners, each with a folder. In each table/model folder, we then have forms, views, templates.
        * Apart from the puppies and owners folders, we also have top level templates folder, __init__.py, models.py, etc. These all will be used by the scripts in either puppies or owners folder. 

### The process of creating the project structure
The following steps are used to create the project structure introduced earlier.  

* According to the structures laid out in the .txt file, do the following steps:
    * The following is for demonstrating the steps. In fact, all the folders and files are already created and populated in the folder 01-Using-Blueprints.
    * Create an app.py file.
    * Create a folder myproject. Note this folder and app.py are under the same folder. 
    * Under the myproject folder, create following folders: owners, puppies, templates (this is the top level templates); then create the following files: __init__.py, models.py. 
    * Under /myproject/owners/, create templates folder, then under /myproject/owners/templates/, create folder owners
    * Under /myproject/puppies/, create templates folder, then under /myproject/puppies/templates, create folder puppies
    * Under /myproject/owners/, create views.py and forms.py, which are in the same level as templates. 
    * Under /myproject/pupplies/, create views.py and forms.py
* After setting up the structure of folders, lets populate the folders with files. 
    * Copy the contents of old models.py to the models.py under the top level templates folder. Note add on top "from my project import db". This indicates we need set up db inside the __init__.py under myproject folder. **Need understand this.** 
    * Under the top level templates folder, create base.html and home.html, can then populate them with our old files. So far we finish the templates files for our models.py. Now lets sync it to the __init__.py file. Populate __init__.py file (see this file under myproject folder. After populating, when we use "from myproject import db" in models.py, it will automatically use the contents of __init__.py. **Review the python notes before about packages and modules.**
    * Populate the /myproject/puppies/forms.py, and populate  /myproject/owners/forms.py. This way, we don't have to confuse which AddForm we are using because they are already in their specific folders. Previously we have to use AddPuppyForm...etc to distinguish. 
    * Under folder myproject/puppies/templates/puppies, add list.html, delete.html, add.html, and populate them. 
    * Under folder myproject/owners/templates/owners, add_owners.html and populate it. 
* We did not change much the files in earlier steps. Next, we need do so. 
    * In the file myproject/owners/views.py, we import many new things and do some other changes. 
    * In the file myproject/puppies/views.py, do the similar things as above. 
    * Then register the blueprints to the __init__.py. 
* Populate the top level app.py, and then run the flask app for the **first time.**
    * In the 'courseNotes\web\Flask\06-Larger-Flask-Applications\01-Using-Blueprints
    * run 'activate myflaskenv'
    * delete the folder migrations and the /myproject/data.sqlite if I run this for the first time. Otherwise if I run 'flask db init', it will complain the migrations folder is already there.
    * run 'flask db init'
    * run 'flask db migrate -m "message"
    * run 'flask db upgrade'
    * run 'python app.py'
    * Note here I did not run 'set FLASK_APP='file1.py'. This is because if we don't set this, then flask will automatically find app.py. If it is a different name, then we need set this. 

* Run the app.py for the second time. 
    * Just run 'python app.py'. The information stored in the database earlier will still be there.

### app.py

In [ ]:
# This is app.py, this is the main file called.
from myproject import app
from flask import render_template

#The sentence above "from myproject import app" amounts to
#importing all the stuff of __init__.py here.
#Then we further attach the follwing. So this app.py is still big.
#It just becomes modular a little bit.
#Similarly the __init__.py is also modularized with the use of Blueprint. otherwise
#it can be very large if we write all the views functions in details right in
#the __init__.py file

@app.route('/')
def index():
    return render_template('home.html')

if __name__ == '__main__':
    app.run(debug=True)


### __init__.py

In [ ]:
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate


app = Flask(__name__)

# Often people will also separate these into a separate config.py file
app.config['SECRET_KEY'] = 'mysecretkey'
basedir = os.path.abspath(os.path.dirname(__file__))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app,db)

# NOTE! These imports need to come after you've defined db, otherwise you will
# get errors in your models.py files. Comments: This is because in the beginning
# of views.py, we import models.py, which need db object.

## Grab the blueprints from the other views.py files for each "app"
from myproject.puppies.views import puppies_blueprint
from myproject.owners.views import owners_blueprint

app.register_blueprint(owners_blueprint,url_prefix="/owners")
app.register_blueprint(puppies_blueprint,url_prefix='/puppies')
#This amounts to making the object app to have another two methods?
#As in app.py, we normally use the following way. Here we achieve the similar effect?
# @app.route('/')
# def index():
#     return render_template('home.html')
# However, the normal way may let the view function here very lengthy. So I guess
# at least one of the purposes of Blueprint is to have Flask application modular.


### views.py for puppies folder

In [ ]:
from flask import Blueprint,render_template,redirect,url_for
from myproject import db
from myproject.puppies.forms import AddForm,DelForm
from myproject.models import Puppy

puppies_blueprint = Blueprint('puppies',
                              __name__,
                              template_folder='templates/puppies')
                              #Figure out the purpose of first arguments 'puppies'
                              #From lecturer: it links it to the puppies folder and views.py file we created, 
                              #then we can assign it the name 'puppies'. See the folder structure under /Flask. 


@puppies_blueprint.route('/add', methods=['GET', 'POST'])
def add():
    form = AddForm()

    if form.validate_on_submit():
        name = form.name.data

        # Add new Puppy to database
        new_pup = Puppy(name)
        db.session.add(new_pup)
        db.session.commit()

        return redirect(url_for('puppies.list'))

    return render_template('add.html',form=form)

@puppies_blueprint.route('/list')
def list():
    # Grab a list of puppies from database.
    puppies = Puppy.query.all()
    return render_template('list.html', puppies=puppies)

@puppies_blueprint.route('/delete', methods=['GET', 'POST'])
def delete():

    form = DelForm()

    if form.validate_on_submit():
        id = form.id.data
        pup = Puppy.query.get(id)
        db.session.delete(pup)
        db.session.commit()

        return redirect(url_for('puppies.list'))
    return render_template('delete.html',form=form)
